In [24]:
import warnings

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SVMSMOTE
from sklearn.feature_selection import SelectKBest, mutual_info_classif

### 2. Загрузим и выведем информацию о тренировочных и тестовых данных 

In [25]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
number_of_elements = 3000 
train_data = train_data.sample(n=number_of_elements,
                                        replace=False) 
test_data.head()

,id,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
0,2258,46,55.65,2688.85,Female,0,No,No,Yes,No,DSL,No,No,No,No,No,Yes,One year,No,Credit card (automatic)
1,3864,48,78.75,3682.45,Male,0,No,Yes,Yes,Yes,DSL,Yes,No,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic)
2,1391,7,69.20,477.55,Female,0,Yes,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic)
3,486,3,75.15,216.75,Female,0,Yes,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic)
4,970,14,19.35,324.8,Male,0,No,No,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check


In [26]:

train_data.head()

,id,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,...,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
2692,1118,31,94.50,2979.2,Male,0,No,No,Yes,No,...,No,Yes,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,0
2665,3649,4,19.95,82.9,Female,0,Yes,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Electronic check,0
2287,2263,51,34.20,1782,Female,0,Yes,Yes,No,No phone service,...,Yes,No,No,Yes,No,No,Two year,No,Bank transfer (automatic),0
1467,3608,72,20.55,1432.55,Male,0,Yes,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Electronic check,0
757,4957,66,110.85,7491.75,Male,1,Yes,No,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,One year,Yes,Electronic check,1


### 3. Предварительная обработка данных 

In [27]:
ids = test_data["id"]
key = 'Response'
train_data

,id,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,...,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
2692,1118,31,94.50,2979.2,Male,0,No,No,Yes,No,...,No,Yes,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,0
2665,3649,4,19.95,82.9,Female,0,Yes,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Electronic check,0
2287,2263,51,34.20,1782,Female,0,Yes,Yes,No,No phone service,...,Yes,No,No,Yes,No,No,Two year,No,Bank transfer (automatic),0
1467,3608,72,20.55,1432.55,Male,0,Yes,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Electronic check,0
757,4957,66,110.85,7491.75,Male,1,Yes,No,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,One year,Yes,Electronic check,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882,2460,2,90.35,190.5,Female,0,Yes,No,Yes,No,...,Yes,Yes,No,No,Yes,No,Month-to-month,No,Electronic check,0
248,5094,5,19.25,86.05,Male,0,No,No,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,0
552,2362,23,84.40,1936.85,Female,0,Yes,Yes,Yes,No,...,Yes,Yes,No,Yes,No,No,Month-to-month,No,Mailed check,0
2986,4975,61,81.05,4747.65,Male,0,Yes,No,Yes,No,...,Yes,Yes,No,Yes,Yes,Yes,Two year,Yes,Mailed check,0


In [28]:

# Преобразование категориальных переменных в числовые
train_data = pd.get_dummies(train_data, drop_first=True)

In [29]:
# Разделение данных на признаки и целевую переменную
key = 'Churn'
x = train_data.drop([key], axis=1)
y = train_data[key]

# Разделение данных на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Теперь x_train_scaled и x_test_scaled готовы для использования в модели

In [30]:
feature_names = list(train_data.columns)[:-1]
sel_mi = SelectKBest(mutual_info_classif, k=8).fit(x, y)
features = list(zip(feature_names, sel_mi.get_support()))
print(features)
useful_features = [i[0] for i in features if i[1] == True]
print(useful_features)
x_new = x[useful_features].copy(deep=True)
x_train_new, x_test_new, y_train_new, y_test_new = train_test_split(x_new, y, test_size=0.3)
x_train_new = scaler.fit_transform(x_train_new)
x_test_new = scaler.transform(x_test_new)

x_new.head()



[('id', False), ('ClientPeriod', True), ('MonthlySpending', True), ('IsSeniorCitizen', False), ('Churn', False), ('TotalSpent_100.2', False), ('TotalSpent_100.25', False), ('TotalSpent_100.35', False), ('TotalSpent_1004.35', False), ('TotalSpent_1005.7', False), ('TotalSpent_1007.9', False), ('TotalSpent_1008.7', False), ('TotalSpent_1011.5', False), ('TotalSpent_1012.4', False), ('TotalSpent_1013.35', False), ('TotalSpent_1013.6', False), ('TotalSpent_1014.25', False), ('TotalSpent_102.75', False), ('TotalSpent_1020.75', False), ('TotalSpent_1021.75', False), ('TotalSpent_1021.8', False), ('TotalSpent_1023.85', False), ('TotalSpent_1023.9', False), ('TotalSpent_1024', False), ('TotalSpent_1025.15', False), ('TotalSpent_1025.95', False), ('TotalSpent_1028.75', False), ('TotalSpent_1028.9', False), ('TotalSpent_1029.35', False), ('TotalSpent_1031.7', False), ('TotalSpent_1032', False), ('TotalSpent_1033', False), ('TotalSpent_1033.9', False), ('TotalSpent_1033.95', False), ('TotalSpent_

,ClientPeriod,MonthlySpending,TotalSpent_205.05,HasMultiplePhoneNumbers_Yes,HasDeviceProtection_Yes,HasOnlineTV_Yes,HasContractPhone_One year,PaymentMethod_Credit card (automatic)
2692,31,94.50,False,False,False,True,False,False
2665,4,19.95,False,False,False,False,False,False
2287,51,34.20,False,False,False,False,False,False
1467,72,20.55,False,False,False,False,False,False
757,66,110.85,False,True,True,True,True,False


### 4. Построение модели

In [31]:
# Определение моделей
models = {
    'Квадратичный Дискриминантный Анализ': QuadraticDiscriminantAnalysis(),
    'Логистическая Регрессия': LogisticRegression(solver='liblinear'),
    'Метод Ближайших Соседей': KNeighborsClassifier(),
    'Наивный Байесовский Классификатор': GaussianNB(),
    'Машина Опорных Векторов': SVC()
}

# Параметры для GridSearchCV
params = {
    'Квадратичный Дискриминантный Анализ': {"reg_param": np.linspace(0, 1, num=101)},
    'Логистическая Регрессия': {"max_iter": [100],
                                "solver": ['sag', 'saga', 'newton-cg'],
                                "n_jobs": [-1]},
    'Метод Ближайших Соседей': {"n_neighbors": list(range(3, 7)),
                                "weights": ["uniform", "distance"],
                                "p": [1, 2, 3, 4]},
    'Наивный Байесовский Классификатор': {},
    'Машина Опорных Векторов': {"kernel": ['poly'],
                                'probability': [True],
                                "class_weight": ["balanced"],
                                "max_iter": [-1]}
}

# Поиск лучших моделей
best_models = {}
for name, model in models.items():
    random_search = GridSearchCV(model, params[name], cv=5, scoring='roc_auc', n_jobs=-1, error_score='raise')
    random_search.fit(x_train_new, y_train_new)
    best_model = random_search.best_estimator_
    best_models[name] = best_model
    print(f"Лучшие параметры для {name}: {random_search.best_params_}")
   

Лучшие параметры для Квадратичный Дискриминантный Анализ: {'reg_param': 0.01}
Лучшие параметры для Логистическая Регрессия: {'max_iter': 100, 'n_jobs': -1, 'solver': 'saga'}


C:\Users\sh764\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Лучшие параметры для Метод Ближайших Соседей: {'n_neighbors': 6, 'p': 2, 'weights': 'uniform'}
Лучшие параметры для Наивный Байесовский Классификатор: {}
Лучшие параметры для Машина Опорных Векторов: {'class_weight': 'balanced', 'kernel': 'poly', 'max_iter': -1, 'probability': True}


In [32]:
print(best_models)

{'Квадратичный Дискриминантный Анализ': QuadraticDiscriminantAnalysis(reg_param=0.01), 'Логистическая Регрессия': LogisticRegression(n_jobs=-1, solver='saga'), 'Метод Ближайших Соседей': KNeighborsClassifier(n_neighbors=6), 'Наивный Байесовский Классификатор': GaussianNB(), 'Машина Опорных Векторов': SVC(class_weight='balanced', kernel='poly', probability=True)}


In [34]:
# Инициализация SVMSMOTE
smote = SVMSMOTE(
    sampling_strategy='auto',  # Балансировка minority класса
    random_state=1,           # Для воспроизводимости
    k_neighbors=5,            # Количество соседей для создания синтетических примеров
    m_neighbors=10,           # Количество соседей для определения границы
    svm_estimator=SVC(probability=True)  # SVM для SVMSMOTE
)

# Применение SMOTE к данным
smoted_x, smoted_y = smote.fit_resample(x_train_new, y_train_new)

# Словарь для хранения результатов
results = {}

# Обучение и оценка моделей
for name, model in best_models.items():
    # Обучение модели на сбалансированных данных
    model.fit(smoted_x, smoted_y)
    
    # Предсказание вероятностей (если модель поддерживает predict_proba)
    if hasattr(model, "predict_proba"):
        y_pred = model.predict_proba(x_test_new)[:, 1]  # Оценка на тестовых данных
    else:
        y_pred = model.decision_function(x_test_new)  # Для моделей без predict_proba
    
    # Оценка ROC-AUC
    score = roc_auc_score(y_test_new, y_pred)
    results[name] = score

# Вывод результатов
print(results)

C:\Users\sh764\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


{'Квадратичный Дискриминантный Анализ': 0.8004081632653061, 'Логистическая Регрессия': 0.8001900607571273, 'Метод Ближайших Соседей': 0.7686742483252844, 'Наивный Байесовский Классификатор': 0.7723975697149088, 'Машина Опорных Векторов': 0.8069231967596198}


In [35]:
best_model_name = max(results, key=results.get)
best_model = best_models[best_model_name]
print("Лучшая модель:", best_model_name)


Лучшая модель: Машина Опорных Векторов


In [39]:
# Применение get_dummies к test_data с учетом train_data
test_data = pd.get_dummies(test_data)

# Убедимся, что test_data имеет те же столбцы, что и train_data
missing_cols = set(train_data.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0  # Добавляем отсутствующие столбцы с нулевыми значениями

# Убедимся, что порядок столбцов совпадает
test_data = test_data[train_data.columns]

# Предсказание на тестовых данных
y_pred = best_model.predict_proba(test_data[useful_features])[:, 1]

# Создание файла submission
submission = pd.DataFrame({'id': ids, 'Churn': y_pred})
submission.to_csv('submission.csv', index=False)
print("Файл submission.csv сохранен.")


C:\Users\sh764\AppData\Local\Temp\ipykernel_16092\1285146123.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data[col] = 0  # Добавляем отсутствующие столбцы с нулевыми значениями
C:\Users\sh764\AppData\Local\Temp\ipykernel_16092\1285146123.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data[col] = 0  # Добавляем отсутствующие столбцы с нулевыми значениями
C:\Users\sh764\AppData\Local\Temp\ipykernel_16092\1285146123.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of callin

Файл submission.csv сохранен.
